В качестве домашнего задания вам предлагается поработать над предсказанием погоды. Файл с данными вы найдете в соответствующей директории. Вам будет доступен датасет weather.csv, ПЕРВЫЕ 75% (shuffle = False) которого нужно взять для обучения, последние 25% - для тестирования.

Требуется построить 4 модели которые будут предсказывать целевую переменную <b>RainTomorrow</b> с помощью:

   1. логистической регрессии [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
   
   2. метода ближайших соседей [sklearn.neighbors](https://scikit-learn.org/stable/modules/neighbors.html)
 
   3. Байесовского классификатора [sklearn.naive_bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
   
   4. логистической регрессии реализованной самостоятельно

Затем следует сравнить результаты моделей (по качеству и времени выполнения) и сделать вывод о том, какая модель и с какими параметрами даёт лучшие результаты.

Не забывайте о том, что работа с признаками играет очень большую роль в построении хорошей модели.

Краткое описание данных:

    Date - Дата наблюдений
    Location - Название локации, в которой расположена метеорологическая станция
    MinTemp - Минимальная температура в градусах цельсия
    MaxTemp - Максимальная температура в градусах цельсия
    Rainfall - Количество осадков, зафиксированных за день в мм
    Evaporation - Так называемое "pan evaporation" класса А (мм) за 24 часа до 9 утра
    Sunshine - Число солнечных часов за день
    WindGustDir - направление самого сильного порыва ветра за последние 24 часа
    WindGustSpeed - скорость (км / ч) самого сильного порыва ветра за последние 24 часа
    WindDir9am - направление ветра в 9 утра

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Загрузка данных
data = pd.read_csv('weather.csv')

In [3]:
# Разделение данных на обучающий и тестовый наборы
train_data, test_data = train_test_split(data, test_size=0.25, shuffle=False)

In [4]:
# Разделение на признаки и целевую переменную
X_train = train_data.drop('RainTomorrow', axis=1)
y_train = train_data['RainTomorrow']

X_test = test_data.drop('RainTomorrow', axis=1)
y_test = test_data['RainTomorrow']

In [5]:
# Преобразование категориальных признаков
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Определение индексов колонок с категориальными признаками
categorical_features = ['Date']

# Создание объекта ColumnTransformer для кодирования категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Преобразование данных
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [6]:
# Вручную подсчитываю некоторую статистику
def manual_report(y_test, y_pred):
    # Количество неверно предсказанных
    error_count = np.count_nonzero(y_test != y_pred)
    # Количество верно предсказанных
    success_count = np.count_nonzero(y_test == y_pred)

    # True Positive
    TP = np.sum((y_pred == 'Yes') & (y_test == 'Yes'))
    # True Negative
    TN = np.sum((y_pred == 'No') & (y_test == 'No'))

    # False Positive
    FP = np.sum((y_pred == 'Yes') & (y_test == 'No'))
    # False Negative
    FN = np.sum((y_pred == 'No') & (y_test == 'Yes'))
    
    # Вывод
    df_Negative = pd.DataFrame({'True Negative': [TN], 'False Negative': [FN], 'Precision': [TN/(TN+FN)],
                               'Precision by total count': [success_count/y_pred.shape[0]]}, index=["No"])
    df_Positive = pd.DataFrame({'True Positive': [TP], 'False Positive': [FP], 'Precision': [TP/(TP+FP)], 
                                'Precision by total count': [error_count/y_pred.shape[0]]}, index=["Yes"])

    print(df_Negative)
    print()
    print(df_Positive)

In [7]:
from sklearn.metrics import classification_report
def train_model(X_train, y_train, X_test, y_test, model, manual=False):
    # Обучение модели на обучающем наборе данных
    model.fit(X_train, y_train)

    # Предсказание целевой переменной для тестового набора данных
    y_pred = model.predict(X_test)

    print('\n\t==============', type(model).__name__, '==============')
    if manual:
        manual_report(y_test, y_pred)
    print('\n\t-------------- Classification report --------------')
    print(classification_report(y_test, y_pred))

## 1.1 Обучим модель методом LogisticRegression

In [8]:
from sklearn.linear_model import LogisticRegression

# Создание объекта модели
model = LogisticRegression()

train_model(X_train, y_train, X_test, y_test, model, manual=True)


	============== LogisticRegression ==============
    True Negative  False Negative  Precision  Precision by total count
No          26180            7135   0.785832                  0.751414

     True Positive  False Positive  Precision  Precision by total count
Yes            532            1702   0.238138                  0.248586

	-------------- Classification report --------------
              precision    recall  f1-score   support

          No       0.79      0.94      0.86     27882
         Yes       0.24      0.07      0.11      7667

    accuracy                           0.75     35549
   macro avg       0.51      0.50      0.48     35549
weighted avg       0.67      0.75      0.69     35549



## 1.2 Обучим модель методом ближайших соседей (neighbors)

In [9]:
from sklearn.neighbors import NearestNeighbors # Почему я вообще могу вызвать метод fit для такого метода, 
# что там тренировать, если всегда высчитывается расстояние заново для каждого элемента. Нужен только predict
from sklearn.neighbors import KNeighborsClassifier

# Создание объекта модели
# model = NearestNeighbors(n_neighbors=2, algorithm='ball_tree') # Выдает ошибку отутствия метода predict - странно
model = KNeighborsClassifier(n_neighbors=1)

train_model(X_train, y_train, X_test, y_test, model, manual=True)


	============== KNeighborsClassifier ==============
    True Negative  False Negative  Precision  Precision by total count
No          22434            5855   0.793029                  0.682045

     True Positive  False Positive  Precision  Precision by total count
Yes           1812            5448   0.249587                  0.317955

	-------------- Classification report --------------
              precision    recall  f1-score   support

          No       0.79      0.80      0.80     27882
         Yes       0.25      0.24      0.24      7667

    accuracy                           0.68     35549
   macro avg       0.52      0.52      0.52     35549
weighted avg       0.68      0.68      0.68     35549



## 1.3 Обучим модель методом Байесовского классификатора (naive_bayes)

In [10]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

train_model(X_train, y_train, X_test, y_test, model)


	============== MultinomialNB ==============

	-------------- Classification report --------------
              precision    recall  f1-score   support

          No       0.79      0.93      0.85     27882
         Yes       0.25      0.08      0.12      7667

    accuracy                           0.75     35549
   macro avg       0.52      0.51      0.49     35549
weighted avg       0.67      0.75      0.70     35549



## 2 Отчеты разных моделей

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

models = [LogisticRegression(), 
          KNeighborsClassifier(n_neighbors=1), 
          MultinomialNB(), 
          BernoulliNB(),
         ]

for model in models:
    train_model(X_train, y_train, X_test, y_test, model)


	============== LogisticRegression ==============

	-------------- Classification report --------------
              precision    recall  f1-score   support

          No       0.79      0.94      0.86     27882
         Yes       0.24      0.07      0.11      7667

    accuracy                           0.75     35549
   macro avg       0.51      0.50      0.48     35549
weighted avg       0.67      0.75      0.69     35549


	============== KNeighborsClassifier ==============

	-------------- Classification report --------------
              precision    recall  f1-score   support

          No       0.79      0.80      0.80     27882
         Yes       0.25      0.24      0.24      7667

    accuracy                           0.68     35549
   macro avg       0.52      0.52      0.52     35549
weighted avg       0.68      0.68      0.68     35549


	============== MultinomialNB ==============

	-------------- Classification report --------------
              precision    recall 

## 1.4 Обучим модель методом логистической регрессии реализованной самостоятельно

# Ниже не мой код

### Реализация логистической регрессии
__Логистическая регрессия__

$$p(y|x) = a(x, \theta) = \sigma(\langle x, \theta \rangle) = \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}$$

In [16]:
import math
import numpy as np
weights = np.array([1, 2, 3])

X =  np.array([[ 1,  1, 1],
               [-1, -2, 1],
               [-1, -2, 2],
               [-2, -2, -3]
              ])

y = np.array([1, 1, 0, 0])

In [58]:
def probability(weights, X):
    z = X @ weights # вычисляем линейную комбинацию
    result = 1 / (1 + np.exp(-z)) # применяем сигмоидную функцию
    return result

prob = probability(weights, X)

labels = np.round(prob).astype(int) # преобразуем вероятности в метки классов
print(labels, prob)

assert type(prob) == np.ndarray, 'Возвращается неверный тип'
assert prob.shape == (X.shape[0],), 'Неверный размер массива'
assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'

[1 0 1 0] [9.97527377e-01 1.19202922e-01 7.31058579e-01 3.05902227e-07]


Функция предсказания метки класса, получает на вход вероятности принадлежности к классу 1 и выдает метки классов $y \in \{0, 1\}$

In [89]:
def binary_class_prediction(weights, X, threshold =.5):
    prob =  probability(weights, X)
    labels = np.round(prob).astype(int) # преобразуем вероятности в метки классов
    return labels

y_pred = binary_class_prediction(weights, X)


assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'
assert y_pred.shape == (X.shape[0],), 'Неверный размер массива'
assert min(y_pred) == 0, 'Функция считается неверно'
assert max(y_pred) == 1, 'Функция считается неверно'

__Функционал качества логистической регрессии__

Запишем правдободовие выборки для меток класса $y \in \{+1, -1\}$ 

$$Likelihood(a, X^\ell) = \prod_{i = 1}^{\ell} a(x_i,\theta)^{[y_i = +1]} (1 - a(x_i, \theta))^{[y_i = -1]} → \operatorname*{max}_{\theta}$$ 

Прологарифмируем правдоподобие выборки и перейдем к задаче минимизации:

$$Q(a, X^\ell) =     -\sum_{i = 1}^{\ell} 
        [y_i = +1] \log a(x_i, \theta)
        +
        [y_i = -1] \log (1 - a(x_i, \theta)) \to \operatorname*{min}_{\theta}$$ 
        
Подставим $a(x, \theta)$ в функцинал качества:

$$ Q(a, X^\ell) = -\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{\exp(-\langle \theta, x_i \rangle)}{1 + \exp(-\langle \theta, x_i \rangle)}
\right)
=\\
=
-\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{1}{1 + \exp(\langle \theta, x_i \rangle)}
\right)
=\\
=
\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) $$
    

Итоговый оптимизируемый функционал качества (logloss), записанный для меток классов $y \in \{+1, -1\}$ и усредненный по выборке

$$Q(a, X^\ell) = \frac{1}{\ell}\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) \to \operatorname*{min}_{\theta}$$

Реализуем его в функции logloss:
#### ! Сказано неверно. У нас классы 0 и 1, для них вероятность определяем по другому

In [92]:
def logloss(weights, X, y): 
    # YOUR CODE HERE
    prob =  probability(weights, X)
    
#     print(np.mean(np.log(1+np.exp(-y))))
#     print(np.mean(np.log(1+np.exp(-y * (X @ weights)))))
#     print(np.mean(np.log(1+np.exp(-y * prob))))
#     print(np.mean(-y * np.log(1 / (1 + np.exp(-X @ weights)) - (1-y) * np.log(1 - 1 / (1 + np.exp(-X @ weights))) )))
#     print(np.mean(-y * np.log(prob) - (1-y) * np.log(1 - prob) ))
    result = np.mean(-y * np.log(prob) - (1-y) * np.log(1 - prob) )
    return result

In [93]:
assert logloss(weights, X, y).round(3) == 0.861, 'Функция считается неверно'

__Алгоритм оптимизации функционала качества. Стохастический градиентный спуск__

<b>Вход: </b> Выборка $X^\ell$, темп обучения $h$

<b>Выход: </b> оптимальный вектор весов $\theta$

1.  Инициализировать веса $\theta$
2.  Инициализировать оценку функционала качества: $Q(a, X^\ell)$
3.  <b>Повторять</b>: 

    Выбрать случайным образом подвыборку объектов $X^{batch} =\{x_1, \dots,x_n \}$ из $X^{\ell}$
    
    Рассчитать градиент функционала качества: $\nabla Q(X^{batch}, \theta)$
    
    Обновить веса: $\theta := \theta - h\cdot \nabla Q(X^{batch}, \theta)$
       
    <b>Пока</b> значение $Q$ и/или веса $\theta$ не сойдутся   

Реализуем функцию рассчета градиента функционала качества

$$\frac{\partial Q(a, X^{batch}) }{\partial \theta_j}   = \frac{\partial \frac{1}{n}\sum_{i = 1}^{n}
    \log \left(
        1 + \exp(- y_i \langle \theta, x_i \rangle)
    \right)} {\partial \theta_j}  = \frac{1}{n}\sum_{i = 1}^{n}
     \frac {1}{
        1 + \exp(- y_i \langle \theta, x_i \rangle)} \cdot  \exp(- y_i \langle \theta, x_i \rangle) \cdot -y_i x_{ij}$$

Реализуйте рассчет градиента в матричном виде:

In [ ]:
def gradient(theta, X, y):
    # YOUR CODE HERE
    
    return result 

assert gradient(theta, X, y).shape == theta.shape, 'Неверный размер массива'

Функция обучения уже реализована

In [ ]:
def fit(X, y, batch_size=10, h=0.05,  iters=100, plot=True):

    # получаем размерности матрицы
    size, dim = X.shape

    # случайная начальная инициализация
    theta = np.random.uniform(size=dim)
    
    errors = []
    
    theta_history = theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
    # plt 
    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')
        
        
    for _ in range(iters):  
        
        # берём случайный набор элементов
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]

        # считаем производные
        grad = gradient(theta, X_batch, y_batch)
        
        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'
        
        
        # Обновляем веса
        
        theta -= grad * h
        
        theta_history = np.vstack((theta_history, theta))
        
        # error
        loss = logloss(theta, X, y)
        errors.append(loss)
        
        if plot:
            ax1.clear()            
            ax1.scatter(range(dim), theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')
            
            
            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')
            
            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()   
            
    return theta

In [ ]:
X, y = make_classification(n_samples=2000)

In [ ]:
optimal_theta = fit(X, y)

In [ ]:
y_pred = binary_class_prediction(optimal_theta, X)